# Creating a portfolio out of Nifty50 Stocks 

In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
df = pd.read_csv("NIFTY-50.csv")
df.head()

,SYMBOL \n,OPEN \n,HIGH \n,LOW \n,PREV. CLOSE \n,LTP \n,CHNG \n,%CHNG \n,VOLUME \n(shares),VALUE,52W H \n,52W L \n,365 D % CHNG \n 17-Jun-2022,30 D % CHNG \n 18-May-2023
0,ADANIENT,"2,435.00","2,435.00","2,396.80","2,401.40","2,407.05",5.65,0.24,"348,013","840,507,077.08","4,190.00","1,017.45",14.58,22.77
1,ADANIPORTS,738,738,734,735.65,735.65,-,-,"107,998","79,437,928.90",987.85,395.1,9.97,6.91
2,APOLLOHOSP,"5,174.95","5,197.90","5,127.35","5,145.00","5,133.00",-12,-0.23,"44,707","231,200,015.15","5,294.00","3,635.20",39.01,15.69
3,ASIANPAINT,"3,315.00","3,320.00","3,301.50","3,309.70","3,303.00",-6.7,-0.2,"44,817","148,516,367.28","3,582.90","2,588.35",28.27,7.30
4,AXISBANK,965.95,968.7,963.9,965.65,964.75,-0.9,-0.09,"292,817","282,808,514.94",989.7,618.25,52.02,4.50


In [3]:
a = []
for i in df.columns:
    a.append(i.strip('\n').strip())
df.columns = a
df.rename(columns = {'VOLUME \n(shares)':'VOLUME'}, inplace = True)

In [4]:
df.head()

,SYMBOL,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CHNG,%CHNG,VOLUME,VALUE,52W H,52W L,365 D % CHNG \n 17-Jun-2022,30 D % CHNG \n 18-May-2023
0,ADANIENT,"2,435.00","2,435.00","2,396.80","2,401.40","2,407.05",5.65,0.24,"348,013","840,507,077.08","4,190.00","1,017.45",14.58,22.77
1,ADANIPORTS,738,738,734,735.65,735.65,-,-,"107,998","79,437,928.90",987.85,395.1,9.97,6.91
2,APOLLOHOSP,"5,174.95","5,197.90","5,127.35","5,145.00","5,133.00",-12,-0.23,"44,707","231,200,015.15","5,294.00","3,635.20",39.01,15.69
3,ASIANPAINT,"3,315.00","3,320.00","3,301.50","3,309.70","3,303.00",-6.7,-0.2,"44,817","148,516,367.28","3,582.90","2,588.35",28.27,7.30
4,AXISBANK,965.95,968.7,963.9,965.65,964.75,-0.9,-0.09,"292,817","282,808,514.94",989.7,618.25,52.02,4.50


In [5]:
df.drop(columns  = ['LTP','CHNG','%CHNG','VALUE','52W H',
                    '52W L','365 D % CHNG \n 17-Jun-2022','30 D % CHNG \n 18-May-2023'], inplace = True)

In [6]:
df.head()

,SYMBOL,OPEN,HIGH,LOW,PREV. CLOSE,VOLUME
0,ADANIENT,"2,435.00","2,435.00","2,396.80","2,401.40","348,013"
1,ADANIPORTS,738,738,734,735.65,"107,998"
2,APOLLOHOSP,"5,174.95","5,197.90","5,127.35","5,145.00","44,707"
3,ASIANPAINT,"3,315.00","3,320.00","3,301.50","3,309.70","44,817"
4,AXISBANK,965.95,968.7,963.9,965.65,"292,817"


In [7]:
df_dic = {}
for i, j in enumerate(df['SYMBOL']):
    b = j + '.csv'
    c = pd.read_csv(b)
    d = []
    for k in c.columns:
        k = k.strip()
        if k =='Date':
            d.append(k)
        else:
            d.append(j + '_' + k)
    c.columns = d
    df_dic[i] = c.drop(columns =[j+ '_series', j + '_HIGH', j + '_LOW', j+'_vwap', j + '_ltp',j+'_vwap',
                       j+'_52W H', j+'_52W L', j+'_VALUE', j+'_No of trades', j+'_Symbol'])[::-1]
    df_dic[i].reset_index(drop = True, inplace = True)

In [8]:
for i in range(1,len(df['SYMBOL'])):
    df_dic[0] = df_dic[0].merge(df_dic[i], on = 'Date')

In [9]:
df_dic[0]['Date'] = pd.to_datetime(df_dic[0]['Date']).dt.date

In [10]:
new_df = df_dic[0].drop(columns = ['Date']).replace({',':''},regex=True).apply(pd.to_numeric,1)
new_df.insert(0, 'Date', df_dic[0]['Date'], True)
new_df.head()

,Date,ADANIENT_OPEN,ADANIENT_PREV. CLOSE,ADANIENT_close,ADANIENT_VOLUME,ADANIPORTS_OPEN,ADANIPORTS_PREV. CLOSE,ADANIPORTS_close,ADANIPORTS_VOLUME,APOLLOHOSP_OPEN,...,ULTRACEMCO_close,ULTRACEMCO_VOLUME,UPL_OPEN,UPL_PREV. CLOSE,UPL_close,UPL_VOLUME,WIPRO_OPEN,WIPRO_PREV. CLOSE,WIPRO_close,WIPRO_VOLUME
0,2020-10-01,299.80,297.10,308.15,6914505.0,345.50,341.75,353.30,3004005.0,2151.0,...,4097.75,406483.0,508.00,502.90,505.65,2462815.0,316.4,313.55,313.05,9257331.0
1,2020-10-05,311.50,308.15,326.60,20009598.0,355.90,353.30,350.35,11896462.0,2095.0,...,4047.15,313282.0,509.50,505.65,507.10,2533261.0,315.2,313.05,333.95,38906392.0
2,2020-10-06,329.40,326.60,327.15,7705557.0,356.00,350.35,362.90,7483578.0,2142.5,...,4136.10,351025.0,511.45,507.10,509.55,3662272.0,335.0,333.95,330.05,27579966.0
3,2020-10-07,325.15,327.15,330.10,8234015.0,362.55,362.90,360.65,4024902.0,2119.7,...,4213.35,1037715.0,511.45,509.55,510.85,4997523.0,327.0,330.05,335.30,17739988.0
4,2020-10-08,329.00,330.10,316.25,4125999.0,359.80,360.65,359.75,3036028.0,2190.0,...,4346.00,801790.0,513.95,510.85,509.40,2814294.0,351.0,335.30,359.45,107270570.0


In [11]:
temp1 = {}
for i in df['SYMBOL']:
    temp1[i + '_Daily Return'] = (new_df[i + '_close']/new_df[i + '_PREV. CLOSE'])-1
df_rtn = pd.DataFrame(temp1)
df_rtn = df_rtn.transpose()
s = df_rtn.std()
m = df_rtn.mean()
volatility = (s**(1/252))*100
sharpe = ((m/s).abs())**(1/252)
new_df['Volatility'] = volatility
new_df['Sharpe_Ratio'] = sharpe

In [12]:
temp2 = {}
for i in df['SYMBOL']:
    temp2[i + '_Daily Value'] = new_df[i + '_close']*new_df[i + '_VOLUME']
df_vl = pd.DataFrame(temp2)
df_vl = df_vl.transpose()
value = df_vl.sum()
new_df['Equity Curve'] = value

In [13]:
new_df.head()

,Date,ADANIENT_OPEN,ADANIENT_PREV. CLOSE,ADANIENT_close,ADANIENT_VOLUME,ADANIPORTS_OPEN,ADANIPORTS_PREV. CLOSE,ADANIPORTS_close,ADANIPORTS_VOLUME,APOLLOHOSP_OPEN,...,UPL_PREV. CLOSE,UPL_close,UPL_VOLUME,WIPRO_OPEN,WIPRO_PREV. CLOSE,WIPRO_close,WIPRO_VOLUME,Volatility,Sharpe_Ratio,Equity Curve
0,2020-10-01,299.80,297.10,308.15,6914505.0,345.50,341.75,353.30,3004005.0,2151.0,...,502.90,505.65,2462815.0,316.4,313.55,313.05,9257331.0,98.506380,0.998353,2.661375e+11
1,2020-10-05,311.50,308.15,326.60,20009598.0,355.90,353.30,350.35,11896462.0,2095.0,...,505.65,507.10,2533261.0,315.2,313.05,333.95,38906392.0,98.474672,0.995964,3.204233e+11
2,2020-10-06,329.40,326.60,327.15,7705557.0,356.00,350.35,362.90,7483578.0,2142.5,...,507.10,509.55,3662272.0,335.0,333.95,330.05,27579966.0,98.454402,0.997108,3.081997e+11
3,2020-10-07,325.15,327.15,330.10,8234015.0,362.55,362.90,360.65,4024902.0,2119.7,...,509.55,510.85,4997523.0,327.0,330.05,335.30,17739988.0,98.395292,0.990193,3.325290e+11
4,2020-10-08,329.00,330.10,316.25,4125999.0,359.80,360.65,359.75,3036028.0,2190.0,...,510.85,509.40,2814294.0,351.0,335.30,359.45,107270570.0,98.448050,0.994916,3.900002e+11


In [26]:
new_df.to_csv('Overall_df.csv')

In [15]:
temp = pd.DataFrame(temp2).transpose()
temp.head()

,0,1,2,3,4,5,6,7,8,9,...,683,684,685,686,687,688,689,690,691,692
ADANIENT_Daily Value,2.130705e+09,6.535135e+09,2.520873e+09,2.718048e+09,1.304847e+09,7.727670e+08,8.061384e+08,4.554332e+08,4.756503e+08,1.523441e+09,...,1.575450e+10,6.153804e+09,9.186125e+09,4.776080e+09,6.969611e+09,5.444673e+09,3.352543e+09,9.306346e+09,2.007675e+10,2.755768e+10
ADANIPORTS_Daily Value,1.061315e+09,4.167925e+09,2.715790e+09,1.451581e+09,1.092211e+09,7.104233e+08,1.296982e+09,8.443475e+08,8.247220e+08,9.787876e+08,...,4.094086e+09,1.583621e+09,1.577155e+09,9.812847e+08,3.392759e+09,2.683087e+09,2.884164e+09,4.768987e+09,5.082521e+09,3.866843e+09
APOLLOHOSP_Daily Value,2.558753e+09,3.391554e+09,1.919955e+09,3.195552e+09,4.270967e+09,4.966840e+09,4.667935e+09,2.116980e+09,3.132117e+09,6.073139e+09,...,1.892019e+09,2.778823e+09,2.027326e+09,1.228719e+09,1.693301e+09,1.528642e+09,1.804021e+09,8.935820e+09,4.146062e+09,1.686624e+09
ASIANPAINT_Daily Value,4.110087e+09,3.153282e+09,6.588491e+09,5.114297e+09,2.981548e+09,3.356707e+09,4.535606e+09,5.407841e+09,3.170340e+09,8.365736e+09,...,1.855999e+09,1.515779e+09,1.802589e+09,3.940845e+09,3.287896e+09,5.216713e+09,2.425229e+09,2.658521e+09,3.266006e+09,1.710834e+09
AXISBANK_Daily Value,9.354676e+09,1.077174e+10,8.717525e+09,9.004839e+09,9.448908e+09,1.312852e+10,9.950384e+09,7.189057e+09,6.590998e+09,1.340939e+10,...,1.399822e+10,1.542857e+10,9.823024e+09,1.262054e+10,7.272768e+09,1.501823e+10,8.451652e+09,1.561321e+10,8.697416e+09,1.064711e+10


In [16]:
nifty = pd.read_csv('^NSEI.csv')
z = pd.to_datetime(nifty['Date']).dt.date
nifty = nifty.drop(columns = ['Date']).replace({',':''},regex=True).apply(pd.to_numeric,1)
nifty.insert(0, 'Date', z, True)
nifty.head()

,Date,Open,High,Low,Close,Previous_close,Volume
0,2020-10-01,11364.45020,11428.59961,11347.04981,11416.95020,11247.55000,447700.0
1,2020-10-05,11487.79981,11578.04981,11452.29981,11503.34961,11416.95020,530600.0
2,2020-10-06,11603.45020,11680.29981,11564.29981,11662.40039,11503.34961,561400.0
3,2020-10-07,11679.25000,11763.04981,11629.34961,11738.84961,11662.40039,513400.0
4,2020-10-08,11835.40039,11905.70020,11791.15039,11834.59961,11738.84961,608800.0


In [17]:
temp3 = {}
temp3['Daily return'] = (nifty['Close']/nifty['Previous_close']) - 1
df_rtn1 = pd.DataFrame(temp3)
s1 = df_rtn1.std()
m1 = df_rtn1.mean()
volatility1 = (s1**(1/252))*100
sharpe1 = ((m1/s1).abs())**(1/252)
df_rtn1.transpose()

,0,1,2,3,4,5,6,7,8,9,...,663,664,665,666,667,668,669,670,671,672
Daily return,0.015061,0.007568,0.013826,0.006555,0.008157,0.006726,0.001406,0.000298,0.003063,-0.024284,...,0.000277,0.00685,-0.004905,-0.003818,0.002052,0.006164,0.002124,-0.003615,0.007379,-0.003748


In [18]:
nifty['Equity Curve'] = nifty['Volume']*nifty['Close']
nifty.head()

,Date,Open,High,Low,Close,Previous_close,Volume,Equity Curve
0,2020-10-01,11364.45020,11428.59961,11347.04981,11416.95020,11247.55000,447700.0,5.111369e+09
1,2020-10-05,11487.79981,11578.04981,11452.29981,11503.34961,11416.95020,530600.0,6.103677e+09
2,2020-10-06,11603.45020,11680.29981,11564.29981,11662.40039,11503.34961,561400.0,6.547272e+09
3,2020-10-07,11679.25000,11763.04981,11629.34961,11738.84961,11662.40039,513400.0,6.026725e+09
4,2020-10-08,11835.40039,11905.70020,11791.15039,11834.59961,11738.84961,608800.0,7.204904e+09


In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from dash import dash_table
import plotly.express as px

<ipython-input-19-9adcb595f53b>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-19-9adcb595f53b>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [20]:
app = dash.Dash(__name__)

In [21]:
app.layout = html.Div(children=[html.H1('Nifty50 Stocks',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                html.P("your inputs"),
                                dcc.Input(id='start_date_of_simulation',
                                          placeholder='sim_start',
                                          type ='date',
                                          value=''),
                                html.Br(),

                                dcc.Input(id='end_date_of_simulation',
                                          placeholder='end_date',
                                          type='date',
                                          value=''),
                                html.Br(),
                                
                                dcc.Input(id='n_days_measure_performance',
                                          placeholder='n_days_measure_perf',
                                          type='number',
                                          value=''),
                                html.Br(),
                                
                                dcc.Input(id='n_top_stocks',
                                          placeholder='top_n_stocks',
                                          type='number',
                                          value=''),
                                html.Br(),
                                
                                dcc.Input(id='initial_equity',
                                          placeholder='in_eq',
                                          type='number',
                                          value=''),
                                html.Br(),
                                html.Br(),

                                html.Div(dcc.Graph(id='line-chart')),
                                html.Div(dash_table.DataTable(id = 'my_table')),
                                html.Div(html.P(id='Top_stocks'))
                               ])

In [22]:
@app.callback(Output(component_id='line-chart', component_property='figure'),
              [Input(component_id='start_date_of_simulation', component_property='value'), 
              Input(component_id='end_date_of_simulation', component_property='value')])
def update_output_div(sim_start, end_date):
    filtered_df = new_df.copy()
    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
    filtered_df = filtered_df[(filtered_df['Date'] >= sim_start) & (filtered_df['Date'] <= end_date)]
    fig = px.line(x=filtered_df['Date'], y=filtered_df['Equity Curve'],
                  title='Investment Growth')
    return fig

In [23]:
@app.callback(Output(component_id='Top_stocks', component_property='children'),
             [Input(component_id='n_days_measure_performance', component_property='value'),
             Input(component_id='n_top_stocks', component_property='value')])
def get_top(n_days_measure_perf, top_n_stocks):
    temp_df = pd.DataFrame({})
    temp_df['Daily Value'] = (temp.iloc[:,-1]/temp.iloc[:,-1*int(n_days_measure_perf)])
    temp_df = temp_df.sort_values(by = 'Daily Value',ascending = False)
    temp_arr = []
    for i in temp_df.iloc[:int(top_n_stocks)].index:
        temp_arr.append(i.rstrip('_Daily Value'))
    return 'Top stocks selected: \n\n{}'.format(temp_arr)

In [24]:
@app.callback(Output(component_id='my_table', component_property='data'),
              [Input(component_id='initial_equity', component_property='value'),
              Input(component_id='end_date_of_simulation', component_property='value')])
def update_output_div(in_eq, end_date):
    filtered_df = new_df.copy()
    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
    filtered_df = filtered_df[filtered_df['Date'] <= end_date]
    index_df = pd.DataFrame({})
    index_df['index'] = ['Equal Alloc Buy Hold', 'Nifty']
    index_df['cagr'] = [(filtered_df['Equity Curve'].iloc[-1]/int(in_eq))**(2/5) - 1, (nifty['Equity Curve'].iloc[-1]/int(in_eq))**(2/5) - 1]
    index_df['Volatility'] = [filtered_df['Volatility'].iloc[-1], float(volatility1.values)]
    index_df['Sharpe_Ratio'] = [filtered_df['Sharpe_Ratio'].iloc[-1], float(sharpe1.values)]
    index_df.set_index('index', inplace = True)
    return index_df.to_dict('rows')

In [25]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jun/2023 22:49:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2023 22:49:33] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2023 22:49:33] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2023 22:49:33] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2023 22:49:33] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2023 22:49:33] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2023 22:49:33] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\dash\dash.py", line 1472, in dispatch
    response.set_d

127.0.0.1 - - [21/Jun/2023 22:49:33] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\dash\dash.py", line 1472, in dispatch
    response.set_d

...

127.0.0.1 - - [21/Jun/2023 22:49:50] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\dash\dash.py", line 1472, in dispatch
    response.set_d

127.0.0.1 - - [21/Jun/2023 22:49:50] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\dash\dash.py", line 1472, in dispatch
    response.set_d

127.0.0.1 - - [21/Jun/2023 22:49:50] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [21/Jun/2023 22:49:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2023 22:49:52] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-24-11dd50698031>:14: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

127.0.0.1 - - [21/Jun/2023 22:49:53] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-24-11dd50698031>:14: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

127.0.0.1 - - [21/Jun/2023 22:49:53] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-24-11dd50698031>:14: FutureWarning:

Using short name for 'orient' is 